In [1]:
from pyteaser import Summarize, SummarizeUrl
from pprint import pprint
import pandas as pd
import numpy as np
import tagui as t
import urllib
import sqlite3
from sqlite3 import Error
from datetime import datetime

In [2]:
def create_connection(db_file):
    """ create a database connection to a SQLite database """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print("DB Connection setup")
    except Error as e:
        print(e)
    finally:
        if conn:
            return conn
        else:
            print("DB Connection failed")


if __name__ == '__main__':
    conn = create_connection("./db/news.db")

DB Connection setup


In [4]:
def getSiteByName(name):
    cur = conn.cursor()
    cur.execute("SELECT * FROM site where name=?",(name,))
    rows = cur.fetchall()
    return rows
site = getSiteByName("Today Online")
site_url = site[0][2]
site_id = site[0][0]

In [ ]:
t.init(visual_automation = True, chrome_browser = True)

In [10]:
t.url(site_url)
t.wait(2)
t.hover('//div[@class="container footer-main"]')
t.wait(6)

True

In [11]:
number_to = t.count('(//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")])')
df_to = pd.DataFrame(index=range(0,number_to), columns = ['Sno', 'Title', 'URL', 'Img_URL', 'Summary'])

t.hover('//div[@class="container footer-main"]')
t.wait(2)

for n in range(1, number_to):
    title=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//div[contains(@class, "article-listing_content")]'.format(n))
    URL_o=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//@href'.format(n))
    URL = "https://www.todayonline.com" + str(URL_o) 
    
    Img_link=t.read('//div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][{}]//img/@src'.format(n))
    
    df_to.iloc[n-1, 0] = n
    df_to.iloc[n-1, 1] = title
    df_to.iloc[n-1, 2] = URL
    df_to.iloc[n-1, 3] = Img_link

df_to

[RPA][ERROR] - cannot find //div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][5]//div[contains(@class, "article-listing_content")]
[RPA][ERROR] - cannot find //div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][5]//@href
[RPA][ERROR] - cannot find //div[@class="col"]/div[contains(@class, "today")]/ul/li[contains(@class, "col-md-12")][5]//img/@src


,Sno,Title,URL,Img_URL,Summary
0,1,"Residents, staff at elderly homes being tested...",https://www.todayonline.com/singapore/resident...,https://www.todayonline.com/sites/default/file...,NaN
1,2,Action to be taken against man who made racial...,https://www.todayonline.com/singapore/action-b...,https://www.todayonline.com/sites/default/file...,NaN
2,3,Graduating students can return to school from ...,https://www.todayonline.com/singapore/students...,/sites/all/themes/weekend/templates/static/img...,NaN
3,4,"932 new Covid-19 cases in Singapore, bringing ...",https://www.todayonline.com/singapore/932-new-...,/sites/all/themes/weekend/templates/static/img...,NaN
4,5,,https://www.todayonline.com,,NaN
5,6,Covid-19: Number of community cases down but S...,https://www.todayonline.com/singapore/covid-19...,/sites/all/themes/weekend/templates/static/img...,NaN
6,7,Covid-19: Govt focusing on housing arrangement...,https://www.todayonline.com/singapore/covid-19...,/sites/all/themes/weekend/templates/static/img...,NaN
7,8,Ships in Singapore waters sound horns to show ...,https://www.todayonline.com/singapore/ships-si...,/sites/all/themes/weekend/templates/static/img...,NaN
8,NaN,NaN,NaN,NaN,NaN


In [13]:
for i in range(0, df_to.shape[0]):
    if not df_to['Img_URL'][i].startswith('http') or df_to['Img_URL'][i]=="":
        df_to['Img_URL'][i]=np.nan
        
df_to.dropna(subset=['Img_URL'], inplace=True, how='any')
df_to= df_to.reset_index(drop=True)
df_to['Sno'] = df_to.index
df_to

AttributeError: 'float' object has no attribute 'startswith'